In [17]:
#@title Enter new drug
import requests
import json

# Drug name query
#@markdown Enter substance names in comma separate form.
new_drug = "Gengraf" #@param {type:"string"}

# Parsing input string into list of search terms
substance_name = new_drug.upper()

# Setting up api_key and api_url
api_key = 'z91dwvBJsnOCwOimqWgagaZqGBC53M69pOKoMNPT'
api_url = 'https://api.fda.gov/drug/label.json'

def query_new(substance_name):

  # Parameters for the query (you can customize these based on your needs)
  query_params = {
      'api_key': api_key,
      'search': f'openfda.brand_name:"{substance_name}"',
      'limit': 1,  # Adjust the limit as needed
  }

  # Make the API request
  response = requests.get(api_url, params=query_params)

  # Check if the request was successful (status code 200)
  if response.status_code == 200:
      # Access the JSON response
      data = response.json()

      # Process and print the data
      result = data.get('results', [])

      newdruginfo = {'Drug': substance_name, "Info": result[0]}

      return newdruginfo

  else:
      print(f"Error: No data for {substance_name}. {response.status_code}, {response.text}")

# newdrug_result is result
newdruginfo = query_new(substance_name)

In [18]:
#@title Enter existing medication
import requests
import json

# Drug name query
#@markdown Enter substance names in comma separate form.
existing_medication = "darunavir,lipitor" #@param {type:"string"}

# Parsing input string into list of search terms
substance_names = existing_medication.upper().split(',')

# Setting up api_key and api_url
api_key = 'z91dwvBJsnOCwOimqWgagaZqGBC53M69pOKoMNPT'
api_url = 'https://api.fda.gov/drug/label.json'

def query_existing(substance_names):
  druglist = []

  for substance_name in substance_names:
    # Parameters for the query (you can customize these based on your needs)
    query_params = {
        'api_key': api_key,
        'search': f'openfda.brand_name:"{substance_name}"',
        'limit': 1,  # Adjust the limit as needed
    }

    # Make the API request
    response = requests.get(api_url, params=query_params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Access the JSON response
        data = response.json()

        # Process and print the data
        result = data.get('results', [])
        druginfo = {'Drug': substance_name, "Info": result[0]}

        druglist.append(druginfo)

    else:
        print(f"Error: No data for {substance_name}. {response.status_code}, {response.text}")
  return druglist

# druglist is a list of existing drugs and their data sheets
druglist = query_existing(substance_names)

In [31]:
#@title Find interactions
def find_active_ings(newdruginfo):
  newdrug_result = newdruginfo['Info']
  # find active ingredients of new drug
  found = False
  try:
    act_ings = newdrug_result['spl_patient_package_insert'][0].lower().split(' active ingredient:')[1].split('inactive ingredient')[0]
    found = True
  except:
    pass
  if not found:
    try:
        act_ings = newdrug_result['spl_patient_package_insert'][0].lower().split(' active ingredients:')[1].split('inactive ingredient')[0]
        found = True
    except:
      pass
  if not found:
    try:
      act_ings = newdrug_result['spl_medguide_table'][0].lower().split('>active ingredient:</content> ')[1].split('<')[0]
      found = True
    except:
      pass
  if not found:
      try:
          act_ings = newdrug_result['spl_medguide_table'][0].lower().split('>active ingredients:</content> ')[1].split('<')[0]
          found = True
      except:
        pass
  if not found:
      try:
          act_ings = newdrug_result['precautions'][0].lower().split('is the active ingredient of')[0].strip().split(' ')[-1]
          found = True
      except:
        pass
  if not found:
    act_ings = newdruginfo['Drug'].lower()


  # parse active ingredients into list
  if ',' in act_ings:
    act_ings = [i.strip() for i in act_ings.split(',')]
  else:
    act_ings = [act_ings.strip()]
  return act_ings

# find active ingredients of new drug
act_ings = find_active_ings(newdruginfo)

# test active ingredients of new drug against existing drugs
safe = True
# search each act_ing
for act_ing in act_ings:
  # search against each existing
  for druginfo in druglist:
    try:
      interactions = druginfo['Info']['drug_interactions']
      if act_ing in interactions[0]:
        print(druginfo['Drug'] + " interacts with the new drug because of the new drug's active ingredient " + act_ing + ".")
        spl_set_id = druginfo['Info']['openfda']['spl_set_id'][0]
        url = f"https://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/{spl_set_id}/spl-doc?hl={act_ing}#section-7.1"
        print("Click link for more info: " + url +'\n')
        safe = False
    except:
      pass
    if safe:
      try:
        interactions = druginfo['Info']['drug_interactions_table']
        if act_ing in interactions[0]:
          print(druginfo['Drug'] + " interacts with the new drug because of the new drug's active ingredient " + act_ing + ".")
          spl_set_id = druginfo['Info']['openfda']['spl_set_id'][0]
          url = f"https://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/{spl_set_id}/spl-doc?hl={act_ing}#section-7.1"
          print("Click link for more info: " + url+'\n')
          safe = False
      except:
        pass

# testing existing drugs' active ing against new drug
for druginfo in druglist:
  act_ings = find_active_ings(druginfo)
  for act_ing in act_ings:
    # test against new drug
    try:
      interactions = newdruginfo['Info']['drug_interactions']
      if act_ing in interactions[0]:
        print(druginfo['Drug'] + " interacts with the new drug because of its active ingredient " + act_ing + ".")
        spl_set_id = newdruginfo['Info']['openfda']['spl_set_id'][0]
        url = f"https://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/{spl_set_id}/spl-doc?hl={act_ing}#section-7.1"
        print("Click link for more info: " + url +'\n')
        safe = False
    except:
      pass
    if safe:
      try:
        interactions = newdruginfo['Info']['drug_interactions_table']
        if act_ing in interactions[0]:
          print(druginfo['Drug'] + " interacts with the new drug because of its active ingredient " + act_ing + ".")
          spl_set_id = newdruginfo['Info']['openfda']['spl_set_id'][0]
          url = f"https://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/{spl_set_id}/spl-doc?hl={act_ing}#section-7.1"
          print("Click link for more info: " + url+'\n')
          safe = False
      except:
        pass

if safe:
  print("No interactions found between existing medication and new drug.")

DARUNAVIR interacts with the new drug because of the new drug's active ingredient cyclosporine.
Click link for more info: https://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/5624cc30-d484-458e-b234-9bb745a118ca/spl-doc?hl=cyclosporine#section-7.1

LIPITOR interacts with the new drug because of the new drug's active ingredient cyclosporine.
Click link for more info: https://nctr-crs.fda.gov/fdalabel/services/spl/set-ids/a60cc18b-0631-4cf0-b021-9f52224ece65/spl-doc?hl=cyclosporine#section-7.1

